#### In June 2017, a heavy monsoon rain triggered a series of floods and landslides in Bangladesh, especially in Rangamati, Chittagong, and Bandarban - three hilly districts of Bangladesh - which resulted in the death of at least 152 people. The landslides were caused by a combination of factors, including incessant downpour, steep slopes, deforestation, and rapid urbanization. A study conducted after the event indicated that the large portion of the area was already highly susceptible to landslide due to existing factors and the intense rainfall acted as a triggering factor. Lack of land and poverty also forced many poor people to live in marginal areas where rents are low, including on landslide-prone hills (Reuters).

#### In this project, I am going to analyze the factors that predict how vulnerable each District (City & its administrative region), Upazila (Sub-Unit of District), and Union(village) in the flood-prone country on an existing datasets released by the Dutch Red Cross representing numerous statistics about a county's geography and determining how vulnerable they are to natural disasters. In addition, I am also going to look at flooding data from all existing geostations in Bangladesh using Geocoding-APIs and a dataset supplied by the Geostations. My ultimate goal is to see how a district/Upazila/Union's elevation and their proximity to rivers water levels affect their vulnerability to being flooded.

##### Link to the datasets: https://data.world/neth-red-cross/d0e90210-f207-4b96-9b40-dffc506387c4/workspace/file?filename=geolocations-stations-xlsx-1.csv

#### Importing necessary modules, will import more as needed.

In [1]:
# importing necessary modules, will import more as needed
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from altair import Chart, X, Y

#### Loading the first dataset. This is from the Dutch Red Cross anazlying the 2017 floods. Some key features taken into account is a village's population, poverty levels, & other factors that may make them more succeptible to floods.

In [3]:
df = pd.read_csv('flood2.csv')
df.head()

,"GEOCODE11,N,19,11","LANDTYPE,C,25","Division,C,24","District,C,24","Upazila,C,32","Union,C,40","UpzCode,N,13,11","Shape_Leng,N,19,11","Shape_Area,N,19,11","population,N,24,15","pov_rat,N,11,4","pov_score,N,24,15","depr_index,N,24,15","depr_score,N,24,15","vul_index,N,24,15"
0,10040913,Land,Barisal,Barguna,Amtali,Amtali,100409,53655.059149,4.115672e+07,27008.580630,59.8274,5.98274,-2.47912,9.291279,7.479175
1,10040915,Land,Barisal,Barguna,Amtali,Arpangashia,100409,37435.117149,2.304258e+07,11379.207653,59.6481,0.39071,-2.47912,9.291279,2.841423
2,10040915,Water,Barisal,Barguna,Amtali,Arpangashia,100409,15674.909627,8.410392e+06,1475.601088,3.9071,0.39071,-2.47912,9.291279,2.841423
3,10040923,Land,Barisal,Barguna,Amtali,Atharagashia,100409,44889.415013,3.441367e+07,24124.323875,58.8735,5.88735,-2.47912,9.291279,7.421223
4,10040947,Land,Barisal,Barguna,Amtali,Chowra,100409,31015.869043,3.231520e+07,24724.627190,57.7630,5.77630,-2.47912,9.291279,7.353258


#### As you can see, the column names are not easily distinguishable, so I will be cleaning them.

In [4]:
dict = {'GEOCODE11,N,19,11': 'Geocode', 'LANDTYPE,C,25': 'Landtype', 'Division,C,24': 'Division', 'District,C,24': 'District',
        'Upazila,C,32': 'Upazila', 'Union,C,40': 'Union', 'UpzCode,N,13,11': 'UpzCode', 'Shape_Leng,N,19,11': 'Land_leng', 
        'Shape_Area,N,19,11': 'Area', 'population,N,24,15': 'Population', 'pov_rat,N,11,4': 'Pov_rate', 'pov_score,N,24,15': 'Pov_score', 
        'depr_index,N,24,15': 'depr_index', 'depr_score,N,24,15': 'depr_score', 'vul_index,N,24,15': 'vul_index'}
df.rename(columns=dict,
          inplace=True)
df.head()

,Geocode,Landtype,Division,District,Upazila,Union,UpzCode,Land_leng,Area,Population,Pov_rate,Pov_score,depr_index,depr_score,vul_index
0,10040913,Land,Barisal,Barguna,Amtali,Amtali,100409,53655.059149,4.115672e+07,27008.580630,59.8274,5.98274,-2.47912,9.291279,7.479175
1,10040915,Land,Barisal,Barguna,Amtali,Arpangashia,100409,37435.117149,2.304258e+07,11379.207653,59.6481,0.39071,-2.47912,9.291279,2.841423
2,10040915,Water,Barisal,Barguna,Amtali,Arpangashia,100409,15674.909627,8.410392e+06,1475.601088,3.9071,0.39071,-2.47912,9.291279,2.841423
3,10040923,Land,Barisal,Barguna,Amtali,Atharagashia,100409,44889.415013,3.441367e+07,24124.323875,58.8735,5.88735,-2.47912,9.291279,7.421223
4,10040947,Land,Barisal,Barguna,Amtali,Chowra,100409,31015.869043,3.231520e+07,24724.627190,57.7630,5.77630,-2.47912,9.291279,7.353258


#### Doing aggregate statistics to get a better sense of the data.

#### Top 10 largest districts (cities & surrounding administrative regions) by area

In [5]:
temp = df.groupby(['District'])[['Area']].sum()
temp.sort_values('Area', ascending = False).head(10)

,Area
District,
Rangamati,5.766180e+09
Chittagong,4.779564e+09
Bandarban,4.598389e+09
Mymensingh,4.345228e+09
Khulna,4.204036e+09
Bagerhat,3.948955e+09
Satkhira,3.890415e+09
Sunamganj,3.692350e+09
Dinajpur,3.458702e+09


#### As we can see, the 2017 floods affected the three largest districts, proving the implications of the flood.

#### Top 10 largest districts by population, which happen to be the major cities of the country as expected.

In [6]:
temp = df.groupby(['District'])[['Population']].sum()
temp.sort_values('Population', ascending = False).head(10)

,Population
District,
Dhaka,1.361212e+07
Chittagong,8.124569e+06
Comilla,5.902557e+06
Mymensingh,5.603930e+06
Tangail,3.963925e+06
Gazipur,3.816004e+06
Sylhet,3.775954e+06
Bogra,3.750549e+06
Noakhali,3.407992e+06


#### Top 10 most vulnerable districts. As you can see, it is hard to make sense of this data by just looking at dataframes, especially if you do not have any prior knowledge about the cities. So, we will move on to visualizations.

In [7]:
temp = df.groupby(['District'])[['vul_index']].mean()
temp.sort_values('vul_index', ascending = False).head(10)

,vul_index
District,
Meherpur,7.854433
Bandarban,7.791367
Rangamati,7.753099
Chuadanga,7.683863
Narail,7.644647
Joypurhat,7.636695
Panchagarh,7.627095
Rajbari,7.605418
Nawabganj,7.563118


#### A part of my experiment also prompted me to explore the correlations of the country's population, poverty, and vulnerable indices, so I thought these small scatterplots would perform the best. As we can see, there is a possitive correlation between every district's poverty level and how that makes them vulnerable to floods.

In [8]:
# doing multiple graphs (all same type)
import altair as alt
temp = df.groupby(['District']).mean()
temp['District']= temp.index
alt.Chart(temp).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'), #set the X axes to call from the "column" list below
    alt.Y(alt.repeat("row"), type='quantitative'), #set the Y axes....
    color='District:N'
).properties(
    width=125,
    height=125
).repeat(
    row=['Population', 'Pov_score', 'vul_index'], 
    column=['Population', 'Pov_score', 'vul_index']
)

alt.RepeatChart(...)

#### This is a bar graph that helps us visualize how vulnerable most of the cities in Bangladesh are to being flooded. Bangladesh, being a delta where three major rivers end, has historically been an attractive region for farming. Most of the cities in the country are situated near rivers, making them more prone to being flooded.

#### The two most developed cities, in the country (Dhaka & Chittagong) have infrastructure to withstand flooding. Unfortunately, the other districts that are less developed are more prone to being flooded.

In [21]:
temp = df.sort_values('Population', ascending = False).head(5000)
alt.Chart(temp).mark_bar().encode(
    y='mean(vul_index):Q', # Q means quantitative
    x='District:N', 
)

alt.Chart(...)

#### To better understand why these cities are so prone to being flooded, let's look at meteorological data, as supplied by the geostations in the country.

In [22]:
gl = pd.read_csv('stations.csv')
gl.drop(columns = ['Unnamed: 8'], axis=1, inplace = True)
gl.head()

,Station Name,River Name,Division,District,Upazilla,Union,Average Land Level,Water Level,Highest Water Level,Danger Level,Longitude,Latitude
0,Chiringa,Matamuhuri,Chittagong,Cox's Bazar,Chakaria,Baraitali,NaN,4.48,7.03,5.80,92.079743,21.773552
1,Lama,Matamuhuri,Chittagong,Cox's Bazar,Chakaria,Kakhara,NaN,8.68,15.46,12.25,92.209356,21.793554
2,Patherghata,Bishkhali River,Barisal,Barguna,Patharghata,Kalmegha,NaN,8.68,3.80,1.85,89.987211,22.050923
3,Dohazari,Sangu,Chittagong,Chittagong,Chandanaish,Hashimpur,NaN,4.98,9.05,7.00,92.063472,22.157910
4,Dasmunia,Tentulia,Barisal,Patuakhali,Dashmina,Rangopaldi,NaN,4.98,NaN,2.59,90.548604,22.230833


#### Again, cleaning the column names. This dataset seems to have previously include Average Land Levels, which are not provided anymore. Fortunately, I will use the Open Topo Data API, which takes a list of latitudes and longitudes, and gives us the elevation of a particular location. This will be super helpful in evaluating whether a citycloser to sea-level is more likely to get flooded or not.

In [23]:
gl['latlong'] = list(zip(gl['Latitude '], gl['Longitude ']))
gl.columns = ['Station Name', 'River Name', 'Division', 'District', 'Upazilla',
       'Union', 'Average Land Level', 'Water Level', 'Highest Water Level',
       'Danger Level', 'Longitude', 'Latitude', 'latlong']
gl.drop(columns = ['Average Land Level'], axis=1, inplace = True)
gl.head()

,Station Name,River Name,Division,District,Upazilla,Union,Water Level,Highest Water Level,Danger Level,Longitude,Latitude,latlong
0,Chiringa,Matamuhuri,Chittagong,Cox's Bazar,Chakaria,Baraitali,4.48,7.03,5.80,92.079743,21.773552,"(21.773552, 92.079743)"
1,Lama,Matamuhuri,Chittagong,Cox's Bazar,Chakaria,Kakhara,8.68,15.46,12.25,92.209356,21.793554,"(21.793554, 92.209356)"
2,Patherghata,Bishkhali River,Barisal,Barguna,Patharghata,Kalmegha,8.68,3.80,1.85,89.987211,22.050923,"(22.050923, 89.987211)"
3,Dohazari,Sangu,Chittagong,Chittagong,Chandanaish,Hashimpur,4.98,9.05,7.00,92.063472,22.157910,"(22.15791, 92.063472)"
4,Dasmunia,Tentulia,Barisal,Patuakhali,Dashmina,Rangopaldi,4.98,NaN,2.59,90.548604,22.230833,"(22.230833, 90.548604)"


In [24]:
from ipyleaflet import Map, Marker, Icon, CircleMarker

In [25]:
from ipywidgets import Layout

In [26]:
defaultLayout=Layout(width='960px', height='960px')

In [40]:
import requests

def get_elevation(latlong):
    html = 'https://api.opentopodata.org/v1/aster30m?locations=' + str(latlong[0]) +"," +str(latlong[1])
    res = requests.get(html)
    status = res.json()['status']
    
    if status == 'INVALID_REQUEST':
        return 9.0 #avg elevation of bangladesh
        
    elevation = res.json()['results'][0]['elevation']
    
    return elevation

In [42]:
gl['Elevation'] = gl['latlong'].map(get_elevation)
gl.head()

,Station Name,River Name,Division,District,Upazilla,Union,Water Level,Highest Water Level,Danger Level,Longitude,Latitude,latlong,Elevation
0,Chiringa,Matamuhuri,Chittagong,Cox's Bazar,Chakaria,Baraitali,4.48,7.03,5.80,92.079743,21.773552,"(21.773552, 92.079743)",10.0
1,Lama,Matamuhuri,Chittagong,Cox's Bazar,Chakaria,Kakhara,8.68,15.46,12.25,92.209356,21.793554,"(21.793554, 92.209356)",11.0
2,Patherghata,Bishkhali River,Barisal,Barguna,Patharghata,Kalmegha,8.68,3.80,1.85,89.987211,22.050923,"(22.050923, 89.987211)",0.0
3,Dohazari,Sangu,Chittagong,Chittagong,Chandanaish,Hashimpur,4.98,9.05,7.00,92.063472,22.157910,"(22.15791, 92.063472)",7.0
4,Dasmunia,Tentulia,Barisal,Patuakhali,Dashmina,Rangopaldi,4.98,NaN,2.59,90.548604,22.230833,"(22.230833, 90.548604)",9.0


#### Now that we are done "filling up" the dataset, we are going to do the same for the initial dataset. If you have noticed, the Unions(villages), that are sub-units of the Upazila (cities), do not have latitudes & longitudes associated with them. Thankfully, I found a Bangladesh-based startup named Barikoi (translating to Where is home?) that provides an API outputting the coordinates of all existing Unions of the country. Thus, I was able to find the coordinates of 4,198 unions in the country. This will be very useful in assigning weather stations to the unions, as there are only 98 existing weather stations in the country.

In [43]:
html = 'http://barikoi.xyz/v1/api/NDcwMjpZUk5TOFZZQUs1/unions'
code = requests.get(html)
code.json()['status']

200

In [87]:
d = {}
places = code['places']
for i in range(len(places)):
    union = places[i]['name']
    coordinates = code['places'][i]['center'][31:-2].split(",")
    long = float(coordinates[0])
    lat = float(coordinates[1])
    lst = [lat, long]
    d[str(union)] = lst

In [88]:
udf = pd.DataFrame.from_dict(d, orient='index', columns = ['latitude', 'longitude'])
udf['Union'] = udf.index
udf.reset_index(inplace = True)
udf.drop('index', axis = 1, inplace = True)
udf.head()

,latitude,longitude,Union
0,24.667967,90.800298,Mashka
1,22.774065,89.263454,Tala
2,23.597621,89.320577,Abaipur
3,23.543116,90.470625,Abdullahpur
4,23.812223,88.976922,Ambaria


#### After successfully making a dataframe from my API queries, I merged it with the original dataset with vulnerability data.

In [89]:
merged = pd.merge(df, udf, on='Union', how='inner')
merged.head()

,Geocode,Landtype,Division,District,Upazila,Union,UpzCode,Land_leng,Area,Population,Pov_rate,Pov_score,depr_index,depr_score,vul_index,latitude,longitude
0,10040913,Land,Barisal,Barguna,Amtali,Amtali,100409,53655.059149,4.115672e+07,27008.580630,59.8274,5.98274,-2.47912,9.291279,7.479175,22.982633,90.030836
1,20467013,Land,Chittagong,Khagrachhari,Matiranga,Amtali,204670,26159.374363,2.557387e+07,4653.684808,60.0178,6.00178,-2.63424,9.386749,7.530077,22.982633,90.030836
2,20840710,Land,Chittagong,Rangamati,Baghai Chhari,Amtali,208407,16702.099212,9.338082e+06,1079.671165,63.1607,6.31607,-3.28644,9.788148,7.886154,22.982633,90.030836
3,30355113,Land,Dhaka,Gopalganj,Kotali Para,Amtali,303551,21515.024560,1.712203e+07,17960.694167,59.3496,5.93496,-1.29074,8.559885,7.143738,22.982633,90.030836
4,10040915,Land,Barisal,Barguna,Amtali,Arpangashia,100409,37435.117149,2.304258e+07,11379.207653,59.6481,0.39071,-2.47912,9.291279,2.841423,22.093208,90.188012


In [90]:
merged['latlong'] = list(zip(merged['latitude'], merged['longitude']))
merged.head()

,Geocode,Landtype,Division,District,Upazila,Union,UpzCode,Land_leng,Area,Population,Pov_rate,Pov_score,depr_index,depr_score,vul_index,latitude,longitude,latlong
0,10040913,Land,Barisal,Barguna,Amtali,Amtali,100409,53655.059149,4.115672e+07,27008.580630,59.8274,5.98274,-2.47912,9.291279,7.479175,22.982633,90.030836,"(22.982632716, 90.030835675)"
1,20467013,Land,Chittagong,Khagrachhari,Matiranga,Amtali,204670,26159.374363,2.557387e+07,4653.684808,60.0178,6.00178,-2.63424,9.386749,7.530077,22.982633,90.030836,"(22.982632716, 90.030835675)"
2,20840710,Land,Chittagong,Rangamati,Baghai Chhari,Amtali,208407,16702.099212,9.338082e+06,1079.671165,63.1607,6.31607,-3.28644,9.788148,7.886154,22.982633,90.030836,"(22.982632716, 90.030835675)"
3,30355113,Land,Dhaka,Gopalganj,Kotali Para,Amtali,303551,21515.024560,1.712203e+07,17960.694167,59.3496,5.93496,-1.29074,8.559885,7.143738,22.982633,90.030836,"(22.982632716, 90.030835675)"
4,10040915,Land,Barisal,Barguna,Amtali,Arpangashia,100409,37435.117149,2.304258e+07,11379.207653,59.6481,0.39071,-2.47912,9.291279,2.841423,22.093208,90.188012,"(22.093207976, 90.188012098)"


#### Time for some visualizations. Here, I am sorting the merged dataframe by the top 1000 unions(villages or towns) that are the most vulnerable to flooding. Using ipyleaflet and our newly acquired latitudes and longitudes, now we can visualize the which areas of the country are the most in danger. 

In [91]:
temp = merged.sort_values('vul_index', ascending = False).head(5000)
temp.head()

,Geocode,Landtype,Division,District,Upazila,Union,UpzCode,Land_leng,Area,Population,Pov_rate,Pov_score,depr_index,depr_score,vul_index,latitude,longitude,latlong
40,10042895,Water,Barisal,Barguna,Barguna Sadar,Naltona,100428,30413.818472,1.053125e+07,29.334407,100.0000,10.0,-2.47912,9.291279,9.639801,22.024364,90.020669,"(22.024363559, 90.020669212)"
4793,10040943,Water,Barisal,Barguna,Amtali,Chhota Bagi,100409,20455.729550,7.674308e+06,14.020017,100.0000,10.0,-2.47912,9.291279,9.639801,22.021861,90.104072,"(22.021860706, 90.104071804)"
39,10042895,Land,Barisal,Barguna,Barguna Sadar,Naltona,100428,31149.145295,3.523540e+07,28804.811698,60.9885,10.0,-2.47912,9.291279,9.639801,22.024364,90.020669,"(22.024363559, 90.020669212)"
32,10042847,Water,Barisal,Barguna,Barguna Sadar,Dhalua,100428,26006.353676,8.042234e+06,37.270935,100.0000,10.0,-2.47912,9.291279,9.639801,23.115392,91.251826,"(23.11539187, 91.251825585)"
31,10042847,Land,Barisal,Barguna,Barguna Sadar,Dhalua,100428,29835.318647,3.184117e+07,26687.395753,60.8798,10.0,-2.47912,9.291279,9.639801,23.115392,91.251826,"(23.11539187, 91.251825585)"


In [93]:
from ipyleaflet import Map, Heatmap
locations = list(temp['latlong'])
dhaka = (23.811056, 90.407608)
bdmap = Map (center=dhaka, zoom=6.5)
heatmap = Heatmap (locations=locations, blur = 0)
bdmap.add_layer(heatmap);
bdmap

Map(center=[23.811056, 90.407608], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

#### As per my hypothesis, here I am visualizing the top 10 flattest cities in the country to see if they are located in these danger areas. As expected, most of them are located in these areas. Given Bangladesh's history as a farm nation, fertile flat lands near the rivers tend to be perferable for poor farmers to settle. Unfortunately, these areas are in the most amount of danger to be flooded.

In [94]:
temp = gl.sort_values('Elevation', ascending = True).head(10)
locations = list(temp['latlong'])
dhaka = (23.811056, 90.407608)

bdmap = Map(center=dhaka, zoom=6.5)
for loc in locations:
    marker = CircleMarker(location=loc, radius=3) # https://ipyleaflet.readthedocs.io/en/latest/layers/circle_marker.html
    bdmap.add_layer(marker)
  
bdmap

Map(center=[23.811056, 90.407608], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [95]:
merged1 = df.merge(gl, left_on=['Union'], right_on=['Union'], how='right')
Chart(merged1).mark_bar().encode(x='Elevation:Q', y='count(vul_index):Q')

alt.Chart(...)

#### Now that the visualizations have supported my argument, I am building a linear regression model to predict how vulnerable parts of the country are to flooding. This is for further exploration behind how we can use data science and existing datasets to accurately predict future floods and evacuate people living in this vulnerable areas to prevent loss of lives. 

In [96]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

#### Splitting the dataset into categorical and numeric variables and one-hot encoding the categorical variables for more accuracy.

In [97]:
cat = gl[['Station Name', 'River Name', 'Division', 'District', 'Upazilla', 'Union']]
num = gl[['Water Level', 'Highest Water Level','Danger Level','Longitude', 'Latitude']]
cat = pd.get_dummies(cat)
new_df = pd.concat([cat, num], axis=1)
new_df.dropna(inplace = True)
X = new_df.drop('Danger Level', axis=1)
y = new_df['Danger Level'].copy()

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [99]:
modelv1 = LinearRegression()
modelv1.fit(X_train, y_train)

LinearRegression()

In [100]:
mean_squared_error(y_test, modelv1.predict(X_test))

0.923138980715827

In [101]:
mean_absolute_error(y_test, modelv1.predict(X_test))

0.7010757236296059

In [102]:
r2_score(y_test, modelv1.predict(X_test)) 

0.9949837822690183

#### The predictions have been spot on. One of the reasons why I think the model worked so well was because I one-hot encoded all of the categorical data and got rid of null values. This is also a relatively small dataset, so predicting Danger Levels was easier. As we get more data in the upcoming years about floods in Bangladesh since unfortunately there has been a rise in floods since 2017 due to climate change, I expect to add more features pertaining to that and build better models. Thank you!

In [ ]:
temp = pd.concat([X_test, y_test], axis = 1)
temp['predictions'] = modelv1.predict(X_test)
Chart(temp).mark_circle().encode(x='Water Level', y='Danger Level') + \
Chart(temp).mark_circle(color='red').encode(x='Water Level', y='predictions')